## Part 1 

In [1]:
import pandas as pd

In [2]:
file = r'../data/Launch Lab Coordiantor Assessment.xlsx'

### Loading spreadsheet 

In [4]:
x = pd.read_excel(file)

In [5]:
print(x.head())

  Candidate ID       Program Placement Status  Days in Pipeline  \
0        C-101  Data Science           Placed                45   
1        C-102     UX Design         Unplaced                92   
2        C-103       Web Dev           Placed                33   
3        C-104  Data Science         Unplaced                67   
4        C-105       Web Dev           Placed                28   

   Interviews Attended  Technical Score (1-10)  Soft Skills Score (1-10)  \
0                    5                     8.5                       7.0   
1                    9                     6.0                       5.5   
2                    3                     9.0                       8.5   
3                    7                     7.5                       6.0   
4                    2                     8.0                       9.0   

  Offer Received Offer Accepted  
0            Yes            Yes  
1             No             No  
2            Yes            Yes  
3   

### Calculate Placement rate

##### From our calculation below, the placement rate per program is at 25% for both Web dev and Data Science and 0% for UX Design

In [6]:
placed_candidates = x[x['Placement Status'] == 'Placed'].groupby('Program').size()
total_candidates = x['Candidate ID'].count()
print(placed_candidates)
print(total_candidates)

Program
Data Science    2
Web Dev         2
dtype: int64
8


In [7]:
placement_rate = (placed_candidates / total_candidates) * 100
print(placement_rate)

Program
Data Science    25.0
Web Dev         25.0
dtype: float64


### Compute average "Days in Pipeline" for placed vs. unplaced candidates

##### The average days in pipeline for unplaced is longer with a difference of 42.5 days

In [8]:
avg_days_in_pipeline = x.groupby('Placement Status')['Days in Pipeline'].mean()
print(avg_days_in_pipeline)

Placement Status
Placed      36.25
Unplaced    78.75
Name: Days in Pipeline, dtype: float64


### Find correlation between Technical Score and Offer Received (use CORREL)

##### We get a postive correlation between Technical Score and Offer Received

In [9]:
x['offer numeric'] = x['Offer Received'].apply(lambda x: 1 if x == 'Yes' else 0)

In [10]:
Correlation = x['Technical Score (1-10)'].corr(x['offer numeric'])
print(Correlation)

0.8754737874097737


### Add column: Interviews per Offer = Interviews Attended / Offers Received (treat 0 offers
as "N/A")

##### Four candidates did not receive any offers 

In [11]:
x['Interviews Per Offer'] = x.apply(lambda row: row['Interviews Attended'] / 1 if row['Offer Received'] == 'Yes' else 'N/A', axis=1)
print(x[['Candidate ID', 'Interviews Per Offer']])


  Candidate ID Interviews Per Offer
0        C-101                  5.0
1        C-102                  N/A
2        C-103                  3.0
3        C-104                  N/A
4        C-105                  2.0
5        C-106                  N/A
6        C-107                  4.0
7        C-108                  N/A


### Flag candidates with Days in Pipeline > 60 AND Placement Status = "Unplaced"

##### Four candidates with id's C-102, C-104, C-106, C-108 were flaged.


In [12]:
flaged_candidates = (x['Days in Pipeline'] > 60) & (x['Placement Status'] == 'Unplaced')
print(x[flaged_candidates]['Candidate ID'])

1    C-102
3    C-104
5    C-106
7    C-108
Name: Candidate ID, dtype: object


### Using IF/AND/OR, create "High Potential" flag:

In [13]:
high_potential_candidates = x[(x['Technical Score (1-10)'] >= 8) & (x['Soft Skills Score (1-10)'] >= 7)]
print(high_potential_candidates[['Candidate ID', 'Placement Status']])

  Candidate ID Placement Status
0        C-101           Placed
2        C-103           Placed
4        C-105           Placed
6        C-107           Placed


### Calculate placement rate for "High Potential" vs. others

##### The placement rate for high potential is 50% vs others

In [14]:
placement_rate_high_potential = high_potential_candidates['Placement Status'].count() / total_candidates * 100
print(placement_rate_high_potential)


50.0


## Part 2

### Load File

In [15]:
file_2 = r'../data/part2.xlsx'

In [16]:
x2 = pd.read_excel(file_2)

In [17]:
x2.head()

,Stage,Drop off rate
0,Screening,0.20
1,Tech Interview,0.35
2,Final Interview,0.25
3,Offer,0.10


### If 100 candidates start, how many reach each stage?